In [1]:
import json
import numpy as np
import numpy.random as rd
from IPython.display import HTML, Javascript, display

In [2]:
def json_numpy_serializer(o):
    if isinstance(o, np.ndarray):
        return o.tolist()
    raise TypeError("{} of type {} is not JSON serializable".format(repr(o), type(o)))

def jsglobal(**params):
    code = [];
    for name, value in params.items():
        jsdata = json.dumps(value, default=json_numpy_serializer)
        code.append("window.{}={};".format(name, jsdata))
    display(Javascript("\n".join(code)))

In [3]:
from covertree.covertree import CoverTree

def l2(p, q):
    return np.sqrt((p-q) @ (p-q))

def extract_node_data(n0, x, y, z, levels, links):
    for k in n0.children.keys():
        for n in n0.children[k]:
            x.append(n.data[0])
            y.append(n.data[1])
            z.append(n.data[2])
            levels.append(k)
            links.append(n0.data)
            links.append(n.data)
            extract_node_data(n, x, y, z, levels, links)

num_pts = 1000
x = np.reshape(rd.randn(3*num_pts), (num_pts, 3))
x = np.apply_along_axis(lambda v: v / np.sqrt(np.sum(v**2)), 1, x)
cube_ct = CoverTree(l2)
for i in range(num_pts):
    cube_ct.insert(x[i, :])
    
x = []
y = []
z = []
r = {}
levels = []
links = []

x.append(cube_ct.root.data[0])
y.append(cube_ct.root.data[1])
z.append(cube_ct.root.data[2])
levels.append(cube_ct.maxlevel)
r['min'] = cube_ct.minlevel
r['max'] = cube_ct.maxlevel

extract_node_data(cube_ct.root, x, y, z, levels, links)

pos = np.vstack([x, y, z]).T
        
jsglobal(POS=pos)
jsglobal(LINKS=links)
jsglobal(R=r)
jsglobal(LEVELS=levels)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
%%javascript

// window.location.reload(true)

// Loading the compiled MathBox bundle.
require.config({
    urlArgs: "", // Make suffix empty
    paths: {
        mathBox: '../tree/static/mathbox/build/mathbox-bundle'
    }
});

// Helper function that setups WebGL context and initializes MathBox.
window.with_mathbox = function(element, func) {
    require(['mathBox'], function(){
        var mathbox = mathBox({
          plugins: ['core', 'controls', 'cursor', 'mathbox'],
          controls: { klass: THREE.OrbitControls },
          mathbox: {inspect: false},
          element: element[0],
          loop: {start: false},
            
        });
        var three = mathbox.three;
        three.renderer.setClearColor(new THREE.Color(0xFFFFFF), 1.0);
        three.camera.position.set(-1, 1, 2);
        three.controls.noKeys = true;
        
        three.element.style.height = "400px";
        three.element.style.width = "100%";
        
        function isInViewport(element) {
          var rect = element.getBoundingClientRect();
          var html = document.documentElement;
          var w = window.innerWidth || html.clientWidth;
          var h = window.innerHeight || html.clientHeight;
          return rect.top < h && rect.left < w && rect.bottom > 0 && rect.right > 0;
        }
        
        // Running update/render loop only for visible plots.
        var intervalId = setInterval(function(){
            if (three.element.offsetParent === null) {
                clearInterval(intervalId);
                three.destroy();
                return;
            }
            var visible = isInViewport(three.canvas);
            if (three.Loop.running != visible) {
                visible? three.Loop.start() : three.Loop.stop();
            }
        }, 100);

        func(mathbox);
        
        window.dispatchEvent(new Event('resize'));
    })
}

<IPython.core.display.Javascript object>

In [5]:
%%javascript
with_mathbox(element, function(mathbox) {
    
    var view = mathbox.cartesian({},{rotation:(t)=>[0, t*0.02, 0]})
      .grid({axes: [1, 3]})
    
    view.array({
        width: LEVELS.length,
      expr: function (emit, i, time) {
        if ((time % (R['max'] - R['min'] + 10))> R['max']-LEVELS[i])
            emit(POS[i][0], POS[i][1], POS[i][2]);
      },
      channels: 3
    });
        
    // Now we can see the data on JS side!
    view.point({color:"#55a", size: 4});
    
    //view.array({width: LINKS.length/2, items: 2, channels: 3, data: LINKS, live: false}).vector({color: 0x4444ff, width: 1});
})

<IPython.core.display.Javascript object>